In [2]:
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [1]:
# Set up some global variables
USE_GPU = True

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models
print_every = 100

print('Using device: ', device)

Using device:  /device:GPU:0


# Preprocessing

In [4]:
# MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

image_size = x_train.shape[1]
original_dim = image_size * image_size

x_train = x_train.astype('float32') / 255. # Nawid - Divides the values by 255
x_test = x_test.astype('float32') / 255.
x_train =np.reshape(x_train, [-1, image_size, image_size, 1])
x_test =np.reshape(x_test, [-1, image_size, image_size, 1])

# Nawid - This encodes the labels as a one-hot vector which is required for the classification 
y_train =tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)
print(y_train[0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


# Architecture - Convolutional Classifier

In [5]:
# conv net parameters
classifier_filters = 64
classifier_kernel = 3
classes = 10

In [6]:
#https://www.tensorflow.org/api_docs/python/tf/keras/Model#-  Model subclassing approach taken

class Conv_classifier(tf.keras.Model):

  def __init__(self,num_filters = classifier_filters,num_kernel_size = classifier_kernel,num_classes = classes):
    super(Conv_classifier, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(filters = num_filters, kernel_size = num_kernel_size, activation ='relu')
    self.conv2 = tf.keras.layers.Conv2D(filters = num_filters//2, kernel_size = num_kernel_size, activation = 'relu')
    self.Flatten = tf.keras.layers.Flatten()
    self.Dense_output = tf.keras.layers.Dense(num_classes, activation = 'softmax')

  def call(self, inputs):
    x = self.conv1(inputs)
    x = self.conv2(x)
    x = self.Flatten(x)
    output_class = self.Dense_output(x)
    return output_class



In [ ]:
MNIST_classifier = Conv_classifier()
#compile model using accuracy to measure model performance
MNIST_classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # Nawid - ADDING THE ACCURACY METRIC Shows the classification accuracy on  the validation set
#train the model
MNIST_classifier.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 0.1341 - accuracy: 0.9607 - val_loss: 0.0538 - val_accuracy: 0.9831
Epoch 2/10
60000/60000 [==============================] - 74s 1ms/sample - loss: 0.0486 - accuracy: 0.9849 - val_loss: 0.0440 - val_accuracy: 0.9855
Epoch 3/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 0.0305 - accuracy: 0.9909 - val_loss: 0.0402 - val_accuracy: 0.9880
Epoch 4/10
60000/60000 [==============================] - 78s 1ms/sample - loss: 0.0206 - accuracy: 0.9934 - val_loss: 0.0412 - val_accuracy: 0.9868
Epoch 5/10
60000/60000 [==============================] - 74s 1ms/sample - loss: 0.0134 - accuracy: 0.9957 - val_loss: 0.0538 - val_accuracy: 0.9863
Epoch 6/10
60000/60000 [==============================] - 77s 1ms/sample - loss: 0.0108 - accuracy: 0.9963 - val_loss: 0.0585 - val_accuracy: 0.9849
Epoch 7/10
60000/60000 [==============================] 

# Architecture- VAE

## Network parameters

In [ ]:
# network parameters
input_shape = (28,28,1)
intermediate_dimension = 64
batch_size = 128
latent_dimension = 2
epochs = 10
conv_filters = 16
conv_kernel_size = 3

classification_results = np.zeros((6,2))

## Encoder architecture

In [ ]:
class Sampling(tf.keras.layers.Layer): # Nawid - Specifies a custom layer
  """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

  def call(self, inputs):
    z_mean, z_log_sigma = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim)) # Nawid - Initalised random values with a mean of 0 and a standard deviation of 1
    return z_mean + tf.exp(0.5 * z_log_sigma) * epsilon

#z = Sampling()((z_mean, z_log_sigma)) # Nawid - Instantiates the custom layer and gets the output of the custom layer which is the value of z

In [ ]:
class Encoder(tf.keras.layers.Layer): # Nawid - This defines the layer for the encoder
  def __init__(self,latent_dim, num_filters = conv_filters, num_kernel_size= conv_kernel_size, intermediate_dim = intermediate_dimension):
    super(Encoder, self).__init__()
    print('Encoder latent_dim',latent_dim)
    self.Conv1 = tf.keras.layers.Conv2D(filters = num_filters, kernel_size = num_kernel_size, activation ='relu', strides = 2, padding = 'same')
    self.Conv2 = tf.keras.layers.Conv2D(filters = 2*num_filters, kernel_size = num_kernel_size, activation ='relu', strides = 2, padding = 'same')
    self.Flatten  = tf.keras.layers.Flatten()

    self.Dense1 = tf.keras.layers.Dense(intermediate_dim, activation='relu')
    self.Dense_mean = tf.keras.layers.Dense(latent_dim)
    self.Dense_log_var = tf.keras.layers.Dense(latent_dim)
    self.sampling = Sampling() # Nawid-  This instantiates the sampling layer for the encoder

  def call(self,inputs):
    x1 = self.Conv1(inputs)
    x1 = self.Conv2(x1)

    self.conv_shape = tf.keras.backend.int_shape(x1) # Nawid - This gets the shape which is required for the decoding when using the conv2D transpose
    flattened_x1 = self.Flatten(x1)

    flattened_intermediate = self.Dense1(flattened_x1)
    z_mean = self.Dense_mean(flattened_intermediate)
    z_log_var = self.Dense_log_var(flattened_intermediate)
    z = self.sampling((z_mean, z_log_var))
    return z_mean,z_log_var, z#, conv_shape # Nawid-  Need to output the z variables as well as the shape for the decoder



## Decoder

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, latent_dim,conv_shape, num_filters = conv_filters, num_kernel_size= conv_kernel_size, intermediate_dim = intermediate_dimension): # Nawid - Need to put the shape of post convolution into the parameters
    super(Decoder,self).__init__()
    print('Decoder latent_dim', latent_dim)
    self.Dense_intermediate = tf.keras.layers.Dense(intermediate_dim, activation='relu')
    self.Dense_original = tf.keras.layers.Dense(conv_shape[1]*conv_shape[2]*conv_shape[3], activation ='relu') # Nawid - Need to use the shape from the encoder
    self.Reshape = tf.keras.layers.Reshape((conv_shape[1], conv_shape[2], conv_shape[3])) # Nawid - Need to reshape to the shape after the convolution
    self.Conv_transpose1 = tf.keras.layers.Conv2DTranspose(filters =num_filters, kernel_size = num_kernel_size, activation ='relu', strides=2, padding='same') # Nawid- Convolutional transpose steps to get back to original shape
    self.Conv_transpose2 = tf.keras.layers.Conv2DTranspose(filters =num_filters//2, kernel_size = num_kernel_size, activation ='relu', strides=2, padding='same')
    self.Conv_transpose3 = tf.keras.layers.Conv2DTranspose(filters =1, kernel_size = num_kernel_size, activation ='sigmoid',padding = 'same')

  def call(self, inputs):
    x1= self.Dense_intermediate(inputs)
    x1 = self.Dense_original(x1)
    x_pre_conv_transpose = self.Reshape(x1)
    x_conv_transpose= self.Conv_transpose1(x_pre_conv_transpose)
    x_conv_transpose = self.Conv_transpose2(x_conv_transpose)
    x_reconstruct = self.Conv_transpose3(x_conv_transpose)
    return x_reconstruct


In [ ]:
def vae_loss(x, x_decoded_mean):
    xent_loss = tf.keras.losses.binary_crossentropy(tf.keras.backend.flatten(x), tf.keras.backend.flatten(x_decoded_mean))
    xent_loss *= original_dim
    #xent_loss = tf.keras.backend.mean(xent_loss)
    kl_loss = - 0.5 * tf.keras.backend.mean(1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var), axis=-1)
    return xent_loss + kl_loss

# Testing classification when there are various different latent dimensions

In [ ]:
for i in range(classification_results.shape[0]):
  x = tf.keras.layers.Input(shape=(image_size,image_size,1))  # Nawid - Input layer
  encoder = Encoder(latent_dimension) # Nawid - Instantiates encoder layer which is required to find the shape property
  z_mean, z_log_var, z = encoder(x)
  shape = encoder.conv_shape
  encoder_model = tf.keras.Model(x,z)
  encoder_model.summary()
  
  output = Decoder(latent_dimension,shape)(z)
  vae =tf.keras.Model(x, output)
  vae.summary()
  
  decoder_input = tf.keras.layers.Input(shape = (latent_dimension))
  decoder_layer = vae.layers[-1]
  decoder_output = decoder_layer(decoder_input)
  
  decoder_model = tf.keras.Model(decoder_input, decoder_output)
  decoder_model.summary()
  vae.compile(optimizer='adam', loss=vae_loss,experimental_run_tf_function=False)

  vae.fit(x_train, x_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, x_test))
  encoded_imgs = encoder_model.predict(x_test)
  decoded_imgs = decoder_model.predict(encoded_imgs)
  #decoded_imgs = vae.predict(x_test_noisy)

  test_loss, test_acc = MNIST_classifier.evaluate(decoded_imgs,  y_test, verbose=2)
  classification_results[i,0] = latent_dimension
  classification_results[i,1] = test_acc
  latent_dimension *= 2
    
print(classification_results)